## Task 1: Dataset Preparation

In [1]:
# E-commerce search queries without spaces (realistic examples)
test_queries = [
    "buysmartphoneonline",       # buy + smartphone + online
    "kidsfashionoffer",          # kids + fashion + offer  
    "applewatchseries6",         # apple + watch + series + 6
    "mensdenimjeanslarge",       # mens + denim + jeans + large
    "womensbluedressdiscount",   # womens + blue + dress + discount
    "laptopapplemacsale",        # laptop + apple + mac + sale
    "redshoeswomenssmall"        # red + shoes + womens + small
]

test_queries

['buysmartphoneonline',
 'kidsfashionoffer',
 'applewatchseries6',
 'mensdenimjeanslarge',
 'womensbluedressdiscount',
 'laptopapplemacsale',
 'redshoeswomenssmall']

In [2]:
# Comprehensive product vocabulary dictionary (25+ words)
product_vocabulary = {
    # Product types
    'smartphone', 'laptop', 'watch', 'jeans', 'dress', 'shoes', 'tablet', 'headphones',
    'shirt', 'pants', 'jacket', 'bag', 'phone', 'computer', 'mouse', 'keyboard',
    
    # Brands
    'apple', 'samsung', 'nike', 'adidas', 'sony', 'dell', 'hp', 'lenovo',
    
    # Sizes
    'small', 'medium', 'large', 'xlarge', 'xxlarge',
    
    # Colors
    'red', 'blue', 'black', 'white', 'green', 'yellow', 'pink', 'purple',
    
    # Categories & descriptors
    'mens', 'womens', 'kids', 'fashion', 'denim', 'cotton', 'leather',
    
    # Common search terms
    'buy', 'online', 'offer', 'discount', 'sale', 'deal', 'cheap', 'best',
    'new', 'used', 'series', 'model', 'pro', 'plus', 'max', 'mini',
    
    # Numbers (common in product names)
    '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'
}

# Convert to sorted list for better performance
product_vocabulary = sorted(product_vocabulary, key=len, reverse=True)
product_vocabulary

['headphones',
 'smartphone',
 'discount',
 'keyboard',
 'computer',
 'xxlarge',
 'samsung',
 'fashion',
 'leather',
 'womens',
 'xlarge',
 'lenovo',
 'medium',
 'tablet',
 'laptop',
 'purple',
 'series',
 'yellow',
 'online',
 'adidas',
 'jacket',
 'cotton',
 'apple',
 'dress',
 'shoes',
 'offer',
 'cheap',
 'green',
 'watch',
 'phone',
 'model',
 'mouse',
 'large',
 'small',
 'shirt',
 'pants',
 'jeans',
 'black',
 'denim',
 'white',
 'sony',
 'dell',
 'blue',
 'best',
 'used',
 'sale',
 'pink',
 'deal',
 'mens',
 'mini',
 'nike',
 'kids',
 'plus',
 'max',
 'red',
 'bag',
 'new',
 'buy',
 'pro',
 '11',
 '12',
 '14',
 '13',
 '10',
 '15',
 'hp',
 '6',
 '8',
 '7',
 '9']

## Task 2: Algorithm Implementation

In [3]:
class QuerySegmenter:
    def __init__(self, vocabulary):
        """
        Initialize the segmenter with a vocabulary.
        Vocabulary is sorted by length (longest first) for greedy matching.
        """
        self.vocabulary = set(vocabulary)
        self.sorted_vocab = sorted(vocabulary, key=len, reverse=True)
    
    def segment_greedy(self, query):
        """
        Greedy segmentation: match longest possible words first.
        
        Args:
            query (str): Input query without spaces
            
        Returns:
            tuple: (segmented_words, unknown_parts)
        """
        query = query.lower()
        segmented = []
        unknown_parts = []
        i = 0
        
        while i < len(query):
            matched = False
            
            # Try to match longest words first
            for word in self.sorted_vocab:
                if query[i:i+len(word)] == word:
                    segmented.append(word)
                    i += len(word)
                    matched = True
                    break
            
            if not matched:
                # Find the next valid word or end of string
                j = i + 1
                while j <= len(query):
                    found_word = False
                    for word in self.sorted_vocab:
                        if j + len(word) <= len(query) and query[j:j+len(word)] == word:
                            found_word = True
                            break
                    if found_word or j == len(query):
                        break
                    j += 1
                
                unknown_parts.append(query[i:j])
                segmented.append(f"[UNKNOWN: {query[i:j]}]")
                i = j
        
        return segmented, unknown_parts
    
    def segment_dynamic(self, query):
        """
        Dynamic programming approach for optimal segmentation.
        Finds segmentation with minimum unknown characters.
        
        Args:
            query (str): Input query without spaces
            
        Returns:
            tuple: (segmented_words, unknown_parts)
        """
        query = query.lower()
        n = len(query)
        
        # dp[i] = (min_unknown_chars, segmentation_list)
        dp = [(float('inf'), [])] * (n + 1)
        dp[0] = (0, [])
        
        for i in range(1, n + 1):
            # Option 1: treat current character as unknown
            if dp[i-1][0] + 1 < dp[i][0]:
                prev_seg = dp[i-1][1].copy()
                if prev_seg and prev_seg[-1].startswith('[UNKNOWN:'):
                    # Extend the last unknown segment
                    prev_seg[-1] = f"[UNKNOWN: {prev_seg[-1][10:-1]}{query[i-1]}]"
                else:
                    # Start new unknown segment
                    prev_seg.append(f"[UNKNOWN: {query[i-1]}]")
                dp[i] = (dp[i-1][0] + 1, prev_seg)
            
            # Option 2: try to match words ending at position i
            for word in self.vocabulary:
                word_len = len(word)
                if i >= word_len and query[i-word_len:i] == word:
                    if dp[i-word_len][0] < dp[i][0]:
                        new_seg = dp[i-word_len][1].copy()
                        new_seg.append(word)
                        dp[i] = (dp[i-word_len][0], new_seg)
        
        segmented = dp[n][1]
        unknown_parts = [seg[10:-1] for seg in segmented if seg.startswith('[UNKNOWN:')]
        
        return segmented, unknown_parts

# Initialize segmenter
segmenter = QuerySegmenter(product_vocabulary)
segmenter

## Task 3: Experimentation and Results

In [4]:
# Experiment with greedy segmentation
greedy_results = {}
for query in test_queries:
    segmented, unknown = segmenter.segment_greedy(query)
    greedy_results[query] = {
        'segmented': segmented,
        'unknown_parts': unknown,
        'success': len(unknown) == 0
    }

greedy_results

{'buysmartphoneonline': {'segmented': ['buy', 'smartphone', 'online'],
  'unknown_parts': [],
  'success': True},
 'kidsfashionoffer': {'segmented': ['kids', 'fashion', 'offer'],
  'unknown_parts': [],
  'success': True},
 'applewatchseries6': {'segmented': ['apple', 'watch', 'series', '6'],
  'unknown_parts': [],
  'success': True},
 'mensdenimjeanslarge': {'segmented': ['mens', 'denim', 'jeans', 'large'],
  'unknown_parts': [],
  'success': True},
 'womensbluedressdiscount': {'segmented': ['womens',
   'blue',
   'dress',
   'discount'],
  'unknown_parts': [],
  'success': True},
 'laptopapplemacsale': {'segmented': ['laptop',
   'apple',
   '[UNKNOWN: mac]',
   'sale'],
  'unknown_parts': ['mac'],
  'success': False},
 'redshoeswomenssmall': {'segmented': ['red', 'shoes', 'womens', 'small'],
  'unknown_parts': [],
  'success': True}}

In [5]:
# Experiment with dynamic programming segmentation
dynamic_results = {}
for query in test_queries:
    segmented, unknown = segmenter.segment_dynamic(query)
    dynamic_results[query] = {
        'segmented': segmented,
        'unknown_parts': unknown,
        'success': len(unknown) == 0
    }

dynamic_results

{'buysmartphoneonline': {'segmented': ['buy', 'smartphone', 'online'],
  'unknown_parts': [],
  'success': True},
 'kidsfashionoffer': {'segmented': ['kids', 'fashion', 'offer'],
  'unknown_parts': [],
  'success': True},
 'applewatchseries6': {'segmented': ['apple', 'watch', 'series', '6'],
  'unknown_parts': [],
  'success': True},
 'mensdenimjeanslarge': {'segmented': ['mens', 'denim', 'jeans', 'large'],
  'unknown_parts': [],
  'success': True},
 'womensbluedressdiscount': {'segmented': ['womens',
   'blue',
   'dress',
   'discount'],
  'unknown_parts': [],
  'success': True},
 'laptopapplemacsale': {'segmented': ['laptop',
   'apple',
   '[UNKNOWN: mac]',
   'sale'],
  'unknown_parts': ['mac'],
  'success': False},
 'redshoeswomenssmall': {'segmented': ['red', 'shoes', 'womens', 'small'],
  'unknown_parts': [],
  'success': True}}

In [6]:
# Test with queries containing unknown words
test_queries_with_unknown = [
    "buyiphonexrcase",          # 'iphone' and 'xr' not in vocabulary
    "nikejordanshoesred",       # 'jordan' not in vocabulary  
    "samsunggalaxytablet",      # 'galaxy' not in vocabulary
    "macbookprolaptopbag",      # 'macbook' and 'pro' (as separate) might cause issues
    "beatsstudioheadphones"     # 'beats' and 'studio' not in vocabulary
]

unknown_word_results = {}
for query in test_queries_with_unknown:
    greedy_seg, greedy_unknown = segmenter.segment_greedy(query)
    dynamic_seg, dynamic_unknown = segmenter.segment_dynamic(query)
    
    unknown_word_results[query] = {
        'greedy': {'segmented': greedy_seg, 'unknown': greedy_unknown},
        'dynamic': {'segmented': dynamic_seg, 'unknown': dynamic_unknown}
    }

unknown_word_results

{'buyiphonexrcase': {'greedy': {'segmented': ['buy',
    '[UNKNOWN: i]',
    'phone',
    '[UNKNOWN: xrcase]'],
   'unknown': ['i', 'xrcase']},
  'dynamic': {'segmented': ['buy',
    '[UNKNOWN: i]',
    'phone',
    '[UNKNOWN: xrcase]'],
   'unknown': ['i', 'xrcase']}},
 'nikejordanshoesred': {'greedy': {'segmented': ['nike',
    '[UNKNOWN: jordan]',
    'shoes',
    'red'],
   'unknown': ['jordan']},
  'dynamic': {'segmented': ['nike', '[UNKNOWN: jordan]', 'shoes', 'red'],
   'unknown': ['jordan']}},
 'samsunggalaxytablet': {'greedy': {'segmented': ['samsung',
    '[UNKNOWN: galaxy]',
    'tablet'],
   'unknown': ['galaxy']},
  'dynamic': {'segmented': ['samsung', '[UNKNOWN: galaxy]', 'tablet'],
   'unknown': ['galaxy']}},
 'macbookprolaptopbag': {'greedy': {'segmented': ['[UNKNOWN: macbook]',
    'pro',
    'laptop',
    'bag'],
   'unknown': ['macbook']},
  'dynamic': {'segmented': ['[UNKNOWN: macbook]', 'pro', 'laptop', 'bag'],
   'unknown': ['macbook']}},
 'beatsstudioheadphones':

In [7]:
# Performance analysis and statistics
def analyze_performance(results_dict, algorithm_name):
    total_queries = len(results_dict)
    successful_segmentations = sum(1 for r in results_dict.values() if r['success'])
    success_rate = (successful_segmentations / total_queries) * 100
    
    total_unknown_parts = sum(len(r['unknown_parts']) for r in results_dict.values())
    avg_unknown_per_query = total_unknown_parts / total_queries if total_queries > 0 else 0
    
    return {
        'algorithm': algorithm_name,
        'total_queries': total_queries,
        'successful_segmentations': successful_segmentations,
        'success_rate_percent': round(success_rate, 2),
        'total_unknown_parts': total_unknown_parts,
        'avg_unknown_per_query': round(avg_unknown_per_query, 2)
    }

# Analyze performance for both algorithms
greedy_stats = analyze_performance(greedy_results, 'Greedy')
dynamic_stats = analyze_performance(dynamic_results, 'Dynamic Programming')

performance_analysis = {
    'greedy_algorithm': greedy_stats,
    'dynamic_programming': dynamic_stats
}

performance_analysis

{'greedy_algorithm': {'algorithm': 'Greedy',
  'total_queries': 7,
  'successful_segmentations': 6,
  'success_rate_percent': 85.71,
  'total_unknown_parts': 1,
  'avg_unknown_per_query': 0.14},
 'dynamic_programming': {'algorithm': 'Dynamic Programming',
  'total_queries': 7,
  'successful_segmentations': 6,
  'success_rate_percent': 85.71,
  'total_unknown_parts': 1,
  'avg_unknown_per_query': 0.14}}

## Analysis and Improvements

In [8]:
# Algorithm behavior analysis and suggested improvements
algorithm_behavior = {
    'greedy_algorithm': {
        'strengths': [
            'Fast and simple implementation',
            'Good performance when vocabulary is comprehensive',
            'Matches longest words first, reducing ambiguity'
        ],
        'weaknesses': [
            'May not find optimal segmentation',
            'Struggles with overlapping word patterns',
            'Cannot backtrack when early choices lead to suboptimal results'
        ],
        'behavior_with_unknown_words': [
            'Identifies unknown segments and marks them clearly',
            'Continues processing after unknown parts',
            'May break single unknown words into multiple segments'
        ]
    },
    'dynamic_programming': {
        'strengths': [
            'Finds optimal segmentation (minimizes unknown characters)',
            'Better handling of overlapping patterns',
            'More robust with partial vocabulary coverage'
        ],
        'weaknesses': [
            'Higher computational complexity O(n²)',
            'More memory intensive',
            'Still limited by vocabulary completeness'
        ],
        'behavior_with_unknown_words': [
            'Minimizes total unknown characters',
            'Consolidates unknown segments when possible',
            'Provides more coherent segmentation overall'
        ]
    }
}

algorithm_behavior

{'greedy_algorithm': {'strengths': ['Fast and simple implementation',
   'Good performance when vocabulary is comprehensive',
   'Matches longest words first, reducing ambiguity'],
  'weaknesses': ['May not find optimal segmentation',
   'Struggles with overlapping word patterns',
   'Cannot backtrack when early choices lead to suboptimal results'],
  'behavior_with_unknown_words': ['Identifies unknown segments and marks them clearly',
   'Continues processing after unknown parts',
   'May break single unknown words into multiple segments']},
 'dynamic_programming': {'strengths': ['Finds optimal segmentation (minimizes unknown characters)',
   'Better handling of overlapping patterns',
   'More robust with partial vocabulary coverage'],
  'weaknesses': ['Higher computational complexity O(n²)',
   'More memory intensive',
   'Still limited by vocabulary completeness'],
  'behavior_with_unknown_words': ['Minimizes total unknown characters',
   'Consolidates unknown segments when possible

In [9]:
# Suggested improvements for handling unknown words
improvement_suggestions = {
    'vocabulary_expansion': [
        'Add common brand names (iPhone, Samsung Galaxy, etc.)',
        'Include product model numbers and versions',
        'Add regional/cultural product terms',
        'Include common misspellings and abbreviations'
    ],
    'fuzzy_matching': [
        'Implement edit distance for similar words',
        'Use phonetic matching (Soundex, Metaphone)',
        'Apply spell correction algorithms',
        'Leverage word embeddings for semantic similarity'
    ],
    'machine_learning_approaches': [
        'Train character-level LSTM/Transformer models',
        'Use statistical n-gram models',
        'Implement conditional random fields (CRFs)',
        'Leverage pre-trained language models (BERT-based)'
    ],
    'hybrid_approaches': [
        'Combine dictionary-based with ML methods',
        'Use confidence scoring for segmentation quality',
        'Implement ensemble methods',
        'Add context-aware segmentation based on e-commerce categories'
    ],
    'real_world_enhancements': [
        'Learn from user search behavior and clicks',
        'Update vocabulary based on trending products',
        'Use product catalog data for domain-specific terms',
        'Implement feedback loops for continuous improvement'
    ]
}

improvement_suggestions

{'vocabulary_expansion': ['Add common brand names (iPhone, Samsung Galaxy, etc.)',
  'Include product model numbers and versions',
  'Add regional/cultural product terms',
  'Include common misspellings and abbreviations'],
 'fuzzy_matching': ['Implement edit distance for similar words',
  'Use phonetic matching (Soundex, Metaphone)',
  'Apply spell correction algorithms',
  'Leverage word embeddings for semantic similarity'],
 'machine_learning_approaches': ['Train character-level LSTM/Transformer models',
  'Use statistical n-gram models',
  'Implement conditional random fields (CRFs)',
  'Leverage pre-trained language models (BERT-based)'],
 'hybrid_approaches': ['Combine dictionary-based with ML methods',
  'Use confidence scoring for segmentation quality',
  'Implement ensemble methods',
  'Add context-aware segmentation based on e-commerce categories'],
 'real_world_enhancements': ['Learn from user search behavior and clicks',
  'Update vocabulary based on trending products',
  '

## Summary

The implemented system successfully demonstrates dictionary-based query segmentation for e-commerce applications:

**Key Achievements:**
- Created comprehensive product vocabulary (50+ terms)
- Implemented two segmentation algorithms (greedy & dynamic programming)
- Tested with realistic e-commerce queries
- Analyzed behavior with unknown words
- Provided detailed improvement suggestions

**Algorithm Performance:**
- Both algorithms handle known vocabulary well
- Dynamic programming provides more optimal segmentation
- Unknown words are properly identified and marked
- System gracefully degrades with incomplete vocabulary

**Real-world Applications:**
This system can be enhanced and deployed for:
- Search query preprocessing
- Product recommendation systems
- Search result ranking improvement
- User intent understanding